<a href="https://www.kaggle.com/renanfioramonte/alexa-nlp?scriptVersionId=88591747" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
review_df = pd.read_csv('../input/amazon-alexa-reviews/amazon_alexa.tsv', sep='\t')

### SUMMARY

In [ ]:
review_df

In [ ]:
review_df.info()

In [ ]:
review_df.describe()

## EDA

In [ ]:
sns.heatmap(review_df.isnull(), cbar=False)

In [ ]:
review_df.isnull().sum()

In [ ]:
review_df.hist(bins=30, figsize=(13,5), color='r')

In [ ]:
review_df['length'] = review_df['verified_reviews'].apply(len)
review_df.head()

In [ ]:
review_df['length'].plot(bins=100, kind='hist')

In [ ]:
review_df.length.describe()

In [ ]:
review_df[review_df['length'] == 2851]['verified_reviews'].iloc[0]

In [ ]:
review_df[review_df['length'] == 1]['verified_reviews'].iloc[0]

#### Creating a df with the POSITIVE reviews

In [ ]:
positive = review_df[review_df['feedback'] == 1]

In [ ]:
positive.describe()

#### Creating a df with the NEGATIVE reviews

In [ ]:
negative = review_df[review_df['feedback'] == 0]

In [ ]:
negative.describe()

In [ ]:
sns.countplot(review_df['feedback'])

In [ ]:
sns.countplot(x = 'rating', data = review_df)

In [ ]:
plt.figure(figsize=(30,15))
sns.barplot(x = 'variation', y = 'rating', data = review_df, palette = 'deep')

In [ ]:
sentences = review_df['verified_reviews'].tolist()

In [ ]:
## OBS.: to create a word cloud, all the strings must be in one variable ie list

In [ ]:
sentences_as_one_string = ' '.join(sentences)

In [ ]:
from wordcloud import WordCloud

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(sentences_as_one_string))

In [ ]:
negative_list = negative['verified_reviews'].tolist()
negative_sentences_as_one_string = ' '.join(negative_list)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(negative_sentences_as_one_string))

### DATA CLEANING

In [ ]:
review_df.head()

In [ ]:
review_df = review_df.drop(['date', 'rating', 'length'], axis = 1)
review_df.head()

### Dummy variables

In [ ]:
review_df['variation'].unique()

In [ ]:
X_cat = review_df[['variation']]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
onehotencoder = OneHotEncoder()

In [ ]:
X_cat = onehotencoder.fit_transform(X_cat).toarray()

In [ ]:
type(X_cat)

In [ ]:
X_cat = pd.DataFrame(X_cat)

In [ ]:
review_df.drop(['variation'], axis = 1, inplace=True)

In [ ]:
review_df.head()

In [ ]:
review_df = pd.concat([review_df, X_cat], axis = 1)
review_df

### REMOVING PUNCTUATION FROM TEXT

In [ ]:
import string
string.punctuation

In [ ]:
test = "You will feel like Tony Stark on this device. I added quite a few Alexa always loads them up quickly. Adding songs that you hear to specific playlists on Amazon Music is also a great feature.I can go on and on and this is only my second day of ownership.I was lucky to buy this for $100 on Prime Day, but I think for $150 is "

In [ ]:
test_punct_removed = [char for char in test if char not in string.punctuation]

In [ ]:
print(test_punct_removed)

In [ ]:
test_punct_removed = ''.join(test_punct_removed)
test_punct_removed

### REMOVING STOP WORDS

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

#### REMOVING STOPWORDS FROM THE DATASET

In [ ]:
test_punct_stop_removed = [word for word in test_punct_removed.split() if word.lower() not in stopwords.words('english')]

In [ ]:
test_punct_stop_removed

### COUNT VECTORIZER

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
sample_data = ['This is the first document.', 'This document is the second document.', 'And this is the third one.', 'Is this the first document?']

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sample_data)

In [ ]:
print(vectorizer.get_feature_names())

In [ ]:
print(X.toarray())

### Cleaning text pipeline

In [ ]:
def msg_cleaning(message):
    text = [char for char in message if char not in string.punctuation]
    text = ''.join(text)
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return text

In [ ]:
#review_df_clean = review_df['verified_reviews'].apply(msg_cleaning)

In [ ]:
#print(review_df_clean[3])

In [ ]:
#print(review_df['verified_reviews'][3])

#### VECTORIZING IN THE NEW DF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer=msg_cleaning)
review_count_vect = vectorizer.fit_transform(review_df['verified_reviews'])

In [ ]:
len(vectorizer.get_feature_names())

### APPLYING TO THE ORIGINAL DF

In [ ]:
review_df.drop(['verified_reviews'], axis = 1, inplace = True)

In [ ]:
review = pd.DataFrame(review_count_vect.toarray())
type(review)

In [ ]:
review_df = pd.concat([review_df, review], axis = 1)
review_df

#### CREATING X AND Y

In [ ]:
X = review_df.drop(['feedback'], axis = 1)
Y = review_df['feedback']

### TRAINING AND ASSESSING METHOD NAIVE BAYES

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, Y_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
Y_pred = naive_bayes.predict(X_test)

In [ ]:
accuracy_score(Y_test, Y_pred)

In [ ]:
cm = confusion_matrix(Y_test, Y_pred)
cm

In [ ]:
sns.heatmap(cm, annot=True)

In [ ]:
print(classification_report(Y_test, Y_pred))

### TRAINING AND ASESSING LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic = LogisticRegression()
logistic.fit(X_train, Y_train)

In [ ]:
Y_pred = logistic.predict(X_test)

In [ ]:
accuracy_score(Y_test, Y_pred)

In [ ]:
print(classification_report(Y_test, Y_pred))

### TESTING THE MODEL IN ONE STRING

In [ ]:
import pickle

In [ ]:
with open('text_classifier_one.pkl', 'wb') as f:
    pickle.dump([logistic, onehotencoder, vectorizer], f)

In [ ]:
with open('text_classifier_one.pkl', 'rb') as f:
    logistic, onehot, vect = pickle.load(f)

In [ ]:
negative.head()

In [ ]:
negative_text = negative.iloc[0:1, [2,3]]
negative_text

In [ ]:
X_cat = negative_text[['variation']]
X_cat

In [ ]:
X_cat = onehot.transform(X_cat).toarray()
X_cat = pd.DataFrame(X_cat)
X_cat

In [ ]:
X_cat.index = negative_text.index

In [ ]:
X_cat.index, negative_text.index

In [ ]:
negative_df = pd.concat([negative_text, X_cat], axis = 1)

In [ ]:
negative_df

In [ ]:
negative_df.drop(['variation'], axis = 1, inplace=True)

In [ ]:
negative_count_vect = vect.transform(negative_df['verified_reviews'])

In [ ]:
review = pd.DataFrame(negative_count_vect.toarray())

In [ ]:
negative_df.drop(['verified_reviews'], axis = 1, inplace=True)
negative_df

In [ ]:
negative_df.index = review.index

In [ ]:
negative_final = pd.concat([negative_df, review], axis = 1)

In [ ]:
negative_final

In [ ]:
logistic.predict(negative_final)

In [ ]:
logistic.predict_proba(negative_final)